In [1]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import osmnx as ox

In [39]:
reader = pd.read_table('../datos/0100922-160910150852091/occurrence.txt',
                           header=0, sep='\t', iterator=True, chunksize=5)

In [ ]:
for i in range(5):
    print(reader.get_chunk())

In [ ]:
reader.get_chunk(1).columns.values

In [ ]:
species_data = reader.get_chunk(10**5)[['decimalLatitude', 'decimalLongitude', 'kingdom', 'phylum', 'class', 'order',
       'family', 'genus', 'subgenus', 'specificEpithet',
       'infraspecificEpithet', 'taxonRank',
       'vernacularName']]

In [ ]:
species_data.head()

In [42]:
geometry = [Point(xy) for xy in zip(species_data.decimalLongitude, species_data.decimalLatitude)]

In [43]:
species_data = gpd.GeoDataFrame(data=species_data, geometry=geometry)

In [ ]:
species_data[:10].plot(marker='.', color='green', markersize=5)

In [11]:
california = ox.gdf_from_place('California')

In [ ]:
california.head()

In [ ]:
california.plot()

In [14]:
species_data.crs = california.crs

In [ ]:
species_data.crs

In [46]:
%%time
species_in_cal = species_data[species_data.intersects(california.geometry[0])]

CPU times: user 5min 31s, sys: 46.4 ms, total: 5min 31s
Wall time: 5min 31s


In [19]:
?species_data.sindex

In [44]:
spatial_index = species_data.sindex

In [ ]:
spatial_index.__class__

In [45]:
%%time
possible_matches_index = list(spatial_index.intersection(california.bounds.iloc[0]))
possible_matches = species_data.iloc[possible_matches_index]
precise_matches = possible_matches[possible_matches.intersects(california.geometry[0])]

CPU times: user 5min 34s, sys: 67 ms, total: 5min 34s
Wall time: 5min 34s


In [ ]:
precise_matches[:10].plot(marker='.', color='green', markersize=5)